In [1]:
import pandas as pd
import os
import json
import xmltodict
import json
import xml.etree.ElementTree as ET

In [2]:
class Load():
    def __init__(self, path):
        self.path = path


    def remove_at_signs(self, obj):
        if isinstance(obj, dict):
            return {key.replace('@', ''): self.remove_at_signs(value) for key, value in obj.items()}
        elif isinstance(obj, list):
            return [self.remove_at_signs(item) for item in obj]
        else:
            return obj
        

    def remove_first_layer(self, json_dict):
        # Extract values from the first layer
        values = list(json_dict.values())
        return values

    def run(self):

        # Path to the XML file
        with open(self.path, 'rb') as f:
            self.xml = f.read()

   
        # open the xml file
        o = xmltodict.parse(self.xml)  # every time you reload the file in colab the key changes (file (1).xml becomes file (2).xml ...)

        json = self.remove_first_layer(self.remove_at_signs(o))[0]

        return json
    


In [3]:
json = Load(f"Demo_rabo\Demo_rabo\Demo_SSIS.dtsx").run()

In [5]:
# find key in json no matter depth

def find_key_in_json(data, target_key):
    """
    Recursively search for a specific key in a JSON object and return its value(s).

    :param data: The JSON object (could be dict or list)
    :param target_key: The key to search for
    :return: A list of values associated with the target key
    """
    results = []

    if isinstance(data, dict):
        for key, value in data.items():
            if key == target_key:
                results.append(value)
            if isinstance(value, (dict, list)):
                results.extend(find_key_in_json(value, target_key))
    elif isinstance(data, list):
        for item in data:
            if isinstance(item, (dict, list)):
                results.extend(find_key_in_json(item, target_key))

    return results


In [6]:
# print control flow nodes and data flow nodes tree


for control_node in json['DTS:Executables']['DTS:Executable']:

    

    print("executable name: ", control_node['DTS:ObjectName'])
    print("executable description: ", control_node['DTS:Description'])

    #print()

    if control_node['DTS:CreationName'] == 'Microsoft.Pipeline': # if the node is a pipeline, then get into data flow components (source and destination)

        for component in control_node['DTS:ObjectData']['pipeline']['components']['component']:
            print()
            print("     component name: ", component['name'])
            print("     component description: ", component['description'])
            #print("     component properties: ", component['properties'])
            print("     component properties: ", find_key_in_json(component, "property"))


            #for property in component['properties']['property']:
#
            #    # !!! remove the ifs, does not depend on source and destination!!!!!
            #    if property['name'] == 'SqlCommand' and component['description'] == 'OLE DB Source': # extract sql command
            #        print("          SQL command: ", property['#text'])
            #    if property['name'] == 'OpenRowset' and component['description'] == 'OLE DB Destination':  # extract source or destination tables (change if)
            #        print("          table name: ", property['#text'])
            #    if property['name'] == 'ParameterMapping' and component['description'] == 'OLE DB Source': # list of variables of sql query
            #        print("          parameters query: ", property['#text'])

    elif control_node['DTS:CreationName'] =='Microsoft.ExecuteSQLTask': # if the task is ExecuteSQL then print query and variables
        print("SQL query: ", control_node['DTS:ObjectData']['SQLTask:SqlTaskData']['SQLTask:SqlStatementSource'])
        try:
            variables = control_node['DTS:ObjectData']['SQLTask:SqlTaskData']['SQLTask:ParameterBinding']
            if type(variables) == list:
                print("Variables: ", [i['SQLTask:DtsVariableName'] for i in control_node['DTS:ObjectData']['SQLTask:SqlTaskData']['SQLTask:ParameterBinding']])
            elif type(variables) == dict:
                print("Variables: ", control_node['DTS:ObjectData']['SQLTask:SqlTaskData']['SQLTask:ParameterBinding']['SQLTask:DtsVariableName'])
        except: 
            pass


    elif control_node['DTS:CreationName'] =='Microsoft.ExpressionTask': 
        print("SQL query: ", control_node['DTS:ObjectData']['ExpressionTask']['Expression'])
    print()


executable name:  Drop insert table
executable description:  Execute SQL Task
SQL query:  DROP TABLE IF EXISTS New_Supp_Prod_output;

executable name:  Merge and filter
executable description:  Data Flow Task

     component name:  Alter name pref
     component description:  Creates new column values by applying expressions to transformation input columns. Create new columns or overwrite existing ones. For example, concatenate the values from the 'first name' and 'last name' column to make a 'full name' column.
     component properties:  [[{'containsID': 'true', 'dataType': 'System.String', 'description': 'Derived Column Expression', 'name': 'Expression', '#text': '"PREF_" + #{Package\\Merge and filter\\Load supplier.Outputs[ODBC Source Output].Columns[CompanyName]}'}, {'containsID': 'true', 'dataType': 'System.String', 'description': 'Derived Column Friendly Expression', 'expressionType': 'Notify', 'name': 'FriendlyExpression', '#text': '"PREF_" + CompanyName'}]]

     component nam

In [11]:
# order nodes in sequence containers

def sort_precedeneces(precedences_list, precedences_list_sorted):

    # for element in precedence list, recursively add all the node one by one in order
    for i in precedences_list:
        # if the last element of the sorted list is the same as the first element of the list of lists
        if precedences_list_sorted[-1] == i[0]:

            precedences_list_sorted.append(i[1]) # append to sorted list
            sort_precedeneces(precedences_list, precedences_list_sorted) # recursively recall the function

    return precedences_list_sorted

            
precedences_list = []

# for precedences in precedence, create list of lists where each list contains the from and to nodes
for constraint in json['DTS:PrecedenceConstraints']['DTS:PrecedenceConstraint']:
    ffrom = f"{constraint['DTS:From']}".split("\\")[-1]
    to = f"{constraint['DTS:To']}".split("\\")[-1]

    precedences_list.append([ffrom, to])

list_1 = [i[1] for i in precedences_list] # list with second elements of all pairs

precedences_list_sorted = []

# for element in precedence list, 
for i in precedences_list:
    # if the first node is never a second node then the node is the first one in the sequence container, therefore append both elements to list, as first and second, then keep adding until done
    if i[0] not in list_1:
        precedences_list_sorted.append(i[0])
        precedences_list_sorted.append(i[1])


print(sort_precedeneces(precedences_list, precedences_list_sorted))
print()

['Set region', 'Merge and filter', 'Drop insert table', 'Overwrite insert table']

